In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.extend([ '../', '../../'])
import torch
from Data import input_data
import dataloader as dl
from args import Args
from model import Het_ConEn, Het_NetEn, EdgePredictor, Het_classify
from smote import oversample
from train import train_smote, test_smote

# Set device to GPU if available, else use CPU
args = Args()
args.yelp()
torch.cuda.empty_cache()

Current device: NVIDIA A100 80GB PCIe


In [2]:
data = torch.load('../data2.pt')
print(data)

HeteroData(
  r={ num_nodes=157499 },
  u={ num_nodes=127228 },
  b={
    num_nodes=5554,
    y=[5554]
  },
  r_embed={ x=[157499, 128] },
  u_embed={ x=[127228, 128] },
  b_embed={ x=[5554, 128] },
  (r, walk, r)={ edge_index=[2, 1536221] },
  (r, walk, u)={ edge_index=[2, 1145847] },
  (r, walk, b)={ edge_index=[2, 222486] },
  (u, walk, r)={ edge_index=[2, 954708] },
  (u, walk, u)={ edge_index=[2, 925690] },
  (u, walk, b)={ edge_index=[2, 160170] },
  (b, walk, r)={ edge_index=[2, 46890] },
  (b, walk, u)={ edge_index=[2, 44549] },
  (b, walk, b)={ edge_index=[2, 8394] }
)


In [3]:
print(data['b','walk','u'].edge_index)
device = args.device
data = data.to(device)

# Send all x tensors to the device
data['r_embed']['x'] = data['r_embed']['x'].to(device)
data['u_embed']['x'] = data['u_embed']['x'].to(device)
data['b_embed']['x'] = data['b_embed']['x'].to(device)

# Send all y tensors to the device
data['b']['y'] = data['b']['y'].to(device)

# Send all edge_index tensors to the device
data['r', 'walk', 'r']['edge_index'] = data['r', 'walk', 'r']['edge_index'].to(device)
data['r', 'walk', 'u']['edge_index'] = data['r', 'walk', 'u']['edge_index'].to(device)
data['r', 'walk', 'b']['edge_index'] = data['r', 'walk', 'b']['edge_index'].to(device)
data['u', 'walk', 'r']['edge_index'] = data['u', 'walk', 'r']['edge_index'].to(device)
data['u', 'walk', 'u']['edge_index'] = data['u', 'walk', 'u']['edge_index'].to(device)
data['u', 'walk', 'b']['edge_index'] = data['u', 'walk', 'b']['edge_index'].to(device)
data['b', 'walk', 'r']['edge_index'] = data['b', 'walk', 'r']['edge_index'].to(device)
data['b', 'walk', 'u']['edge_index'] = data['b', 'walk', 'u']['edge_index'].to(device)
data['b', 'walk', 'b']['edge_index'] = data['b', 'walk', 'b']['edge_index'].to(device)

edge_indices = [ data['b', 'walk', 'b'].edge_index, data['b', 'walk', 'u'].edge_index, data['b', 'walk', 'r'].edge_index ]

tensor([[     0,      0,      0,  ...,   5553,   5553,   5553],
        [114587,  19978, 126091,  ...,  56388, 115916,  14130]])


In [4]:
print(data['b_embed']['x'].dtype)

torch.float32


In [5]:
c_train_num = dl.train_num(data['b'].y, args.im_class_num, args.class_samp_num[0], args.im_ratio)
print(c_train_num, sum(c_train_num))
train_idx, val_idx, test_idx, c_num_mat = dl.segregate(data['b'].y, c_train_num, args.seed[0], args)
print("train_idx: ", train_idx, "\n", len(train_idx))
print("val_idx: ", val_idx, "\n", len(val_idx))
print("test_idx: ", test_idx, "\n", len(test_idx))
# print(c_num_mat)

[50, 500, 500] 1050
0 1919
1 1824
2 1811
train_idx:  [1748, 2059, 1461, 473, 544, 3870, 1591, 4706, 1500, 4499, 5285, 5307, 5011, 924, 4768, 4163, 182, 2780, 5342, 4631, 2129, 852, 5329, 1252, 5136, 2380, 2569, 5142, 3363, 3040, 4133, 4331, 4301, 4126, 3493, 3284, 1620, 423, 1134, 4218, 2624, 3754, 1557, 1011, 3358, 3656, 3042, 821, 175, 1251, 2517, 3879, 2513, 4265, 1728, 2374, 2104, 328, 1814, 4464, 1952, 3833, 5531, 4293, 246, 4341, 1515, 1018, 962, 3050, 666, 3115, 2769, 2010, 1416, 1035, 2117, 5176, 1672, 2473, 3698, 4557, 5295, 1013, 3998, 4587, 361, 2054, 245, 3444, 5300, 4326, 2106, 3128, 1406, 1123, 1311, 136, 4971, 3126, 4350, 4561, 994, 4186, 5303, 4370, 1057, 4904, 5507, 1677, 5049, 261, 1892, 1495, 3614, 2515, 78, 4117, 2606, 5208, 5294, 2028, 1770, 1291, 3034, 1543, 4588, 284, 742, 701, 5393, 1897, 4004, 272, 3056, 3482, 5293, 1130, 5164, 2557, 2595, 2545, 3645, 2982, 1041, 3508, 2300, 1781, 1175, 4358, 2389, 3815, 3275, 645, 92, 3888, 3309, 1310, 169, 314, 757, 4345, 323

In [ ]:
encoder1 = Het_ConEn(args.embed_dim, args.dropout)
encoder2 = Het_NetEn(args.embed_dim, args.dropout)
# encoder1 = torch.load('pretrained/encoder1.pth')
# encoder2 = torch.load('pretrained/encoder2.pth')
classifier = Het_classify(args.embed_dim, args.nclass, args.dropout)
decoder_b = EdgePredictor(args.embed_dim)
decoder_u = EdgePredictor(args.embed_dim)
decoder_r = EdgePredictor(args.embed_dim)
# decoder_b = torch.load('pretrained/decoder_b.pth')
# decoder_u = torch.load('pretrained/decoder_u.pth')
# decoder_r = torch.load('pretrained/decoder_r.pth')

decoder_list = [decoder_b, decoder_u, decoder_r]

encoder1.to(device)
encoder2.to(device)
classifier.to(device)
for decoder in decoder_list:
    decoder.to(device)

## Training Part

In [ ]:
torch.cuda.empty_cache()
train_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, 
            train_idx, val_idx, test_idx, args, os_mode = 'reweight', train_mode = '')

In [ ]:
# test_smote(data, edge_indices, encoder1, encoder2, classifier, decoder_list, test_idx, args = args, dataset = 'Test', os_mode = 'no', train_mode = '')

In [ ]:
# torch.save(decoder_list[0], 'pretrained/decoder_b.pth')
# torch.save(decoder_list[1], 'pretrained/decoder_u.pth')
# torch.save(decoder_list[2], 'pretrained/decoder_r.pth')
# torch.save(encoder1, 'pretrained/encoder1.pth')
# torch.save(encoder2, 'pretrained/encoder2.pth')